# Digital Twin v2.3: Deep History Model

**Author:** Kian Mansouri Jamshidi
**Project Director:** Kian Mansouri Jamshidi
**Date:** 2025-09-27

## Objective
This notebook is our final attempt to maximize the performance of our Digital Twin. Building on the success of the V2 model (R² ≈ 0.71), we will test the hypothesis that a more powerful model can be trained by providing it with a **deeper historical context** and configuring the LightGBM algorithm for **maximum learning capacity**.

### 1. Imports and Data Preparation

In [1]:
import pandas as pd
import numpy as np
import glob
import joblib
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import lightgbm as lgb

sns.set_theme(style="whitegrid")

# --- Load Data ---
PROJECT_ROOT = Path('.').resolve().parent
TELEMETRY_DIR = PROJECT_ROOT / 'data' / 'telemetry_v2'
ARTIFACT_DIR = PROJECT_ROOT / 'artifacts' / 'phase2'
df_list = [pd.read_parquet(file) for file in glob.glob(str(TELEMETRY_DIR / "*.parquet"))]
df = pd.concat(df_list, ignore_index=True).sort_values(by='timestamp').reset_index(drop=True)
if 'cpu_temp_celsius_avg' in df.columns:
    df = df.drop('cpu_temp_celsius_avg', axis=1)
print(f"Data loaded with {len(df)} total points.")

Data loaded with 6928 total points.


### 2. Deep Feature Engineering

We will significantly increase the window and lag sizes to give the model a much longer-term memory of system behavior.

In [2]:
# --- DEEPER HISTORY PARAMETERS --- #
WINDOW_SIZE = 30 # Approx 3 seconds of history for rolling stats
LAG_AMOUNT = 20  # Approx 2 seconds of history for lag features

print(f"Using WINDOW_SIZE={WINDOW_SIZE} and LAG_AMOUNT={LAG_AMOUNT}")

df_featured = df.copy()
workload_dummies = pd.get_dummies(df_featured['workload_type'], prefix='workload')
df_featured = pd.concat([df_featured, workload_dummies], axis=1)

df_featured['overall_util_rolling_mean'] = df_featured['cpu_util_overall'].rolling(window=WINDOW_SIZE).mean()
df_featured['overall_util_rolling_std'] = df_featured['cpu_util_overall'].rolling(window=WINDOW_SIZE).std()

other_core_cols = [col for col in df.columns if 'cpu_util_core' in col and col != 'cpu_util_core_0']
for i in range(1, LAG_AMOUNT + 1):
    df_featured[f'overall_util_lag_{i}'] = df_featured['cpu_util_overall'].shift(i)
    # We will only lag the *other* cores for a shorter period to keep the feature count manageable
    if i <= 10:
        for core_col in other_core_cols:
            df_featured[f'{core_col}_lag_{i}'] = df_featured[core_col].shift(i)

df_model = df_featured.drop('workload_type', axis=1).dropna().reset_index(drop=True)

target = 'cpu_util_core_0'
features = [col for col in df_model.columns if ('cpu_util' in col and col != target) or 'workload_' in col]

X = df_model[features]
y = df_model[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Feature engineering complete. Training set size: {len(X_train)}")
print(f"Total deep features: {len(features)}")

Using WINDOW_SIZE=30 and LAG_AMOUNT=20
Feature engineering complete. Training set size: 5519
Total deep features: 83


### 3. Train High-Capacity Model

We will now train a LightGBM model configured for higher accuracy and learning capacity.

In [3]:
# --- HIGH-CAPACITY MODEL PARAMETERS --- #
deep_params = {
    'objective': 'regression_l1',
    'n_estimators': 2000,        # More trees
    'learning_rate': 0.01,       # Slower learning
    'num_leaves': 50,            # More complex trees
    'colsample_bytree': 0.7,
    'subsample': 0.7,            # Add data subsampling for robustness
    'reg_alpha': 0.1,            # L1 regularization
    'reg_lambda': 0.1,           # L2 regularization
    'random_state': 42,
    'n_jobs': -1
}

model = lgb.LGBMRegressor(**deep_params)

print("Training final Deep History LightGBM model...")
model.fit(
    X_train, 
    y_train,
    eval_set=[(X_test, y_test)],
    eval_metric='r2',
    callbacks=[lgb.early_stopping(100, verbose=True)]
)
print("Model training complete.")

Training final Deep History LightGBM model...
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.012126 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2230
[LightGBM] [Info] Number of data points in the train set: 5519, number of used features: 83
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[1975]	valid_0's l1: 2.79353
Model training complete.


### 4. Final Evaluation

In [4]:
# FINAL EVALUATION CELL (CORRECTED TO ALWAYS SAVE)

y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)

print(f"--- Final Deep Model Performance ---")
print(f"R-squared (R²): {r2:.4f}")

if r2 >= 0.85:
    print("\nMISSION ACCOMPLISHED: Performance has broken the 85% barrier!")
elif r2 > 0.71:
    print("\nBREAKTHROUGH: The deep history features provided a significant performance boost.")
else:
    print("\nLIMIT REACHED: The deep history features did not improve performance. V2.0 remains the champion.")

# --- THE FIX IS HERE: We will now save the model regardless of the score ---
model_path = ARTIFACT_DIR / 'digital_twin_v2.3_deep.joblib'
joblib.dump(model, model_path)
print(f"\nDeep model saved for ensemble experiment to: {model_path}")

--- Final Deep Model Performance ---
R-squared (R²): 0.6979

LIMIT REACHED: The deep history features did not improve performance. V2.0 remains the champion.

Deep model saved for ensemble experiment to: /home/kian/Desktop/ForgeX4-COSMOS-Omega/artifacts/phase2/digital_twin_v2.3_deep.joblib
